# Investigação da consistência dos dados

Ao iniciar uma análise das despesas da pandemia do COVID-19 reparamos que os dados
tinham alguns problemas. Algumas entradas estavam duplicadas e outras não batiam.

Essa análise visa comparar os dados apresentados no Tribunal de Contas dos Municípios da Bahia
(TCM-BA) e o Portal da Transparência da Prefeitura de Feira de Santana.

## Dados utilizados

Para avaliar os dados escolhemos o mês de Julho de 2020 (escolhemos aleatoriamente mas poderia ser qualquer um)
e selecionamos os **pagamentos** da **Prefeitura Municipal de Feira de Santana**.

* [Portal da Transparência da Prefeitura de Feira de Santana](http://www.transparencia.feiradesantana.ba.gov.br/index.php?view=despesa)
* [Tribunal de Contas dos Municípios da Bahia](https://www.tcm.ba.gov.br/consulta-de-despesas)

Exportamos esses dados para XLS e CSV, respectivamente.

## Vamos a análise!

In [65]:
import pandas as pd


cityhall_df = pd.read_excel("../prefeitura-despesas-prefeitura-julho-2020.csv.xls")
tcm_df = pd.read_json("../tcmba-despesas-prefeitura-julho-2020.json")

Uma amostra dos dados da prefeitura:

In [66]:
cityhall_df.sample(3)

,Data de Publicacao,Fase,Numero,N do processo,Bem / Servico Prestado,Credor,CPF / CNPJ,Valor,Funcao,Subfuncao,Natureza,Fonte,Modalidade
617,23/07/2020,PAGAMENTO,08785-20,237-2019,REF A DESPESA COM SERVIÇO DE PUBLICIDADE E PRO...,A.C.PROPAGANDA LTDA. ...,02.811.243/0001-35,"596,08",24 - COMUNICACOES ...,131 - COMUNICACAO SOCIAL ...,339039010000 - Publicidades e Propaganda ...,0000 - RECURSOS ORDINARIOS ...,CONCORRENCIA
788,23/07/2020,PAGAMENTO,6509,,REFERENTE A VIAGEM A SALVADOR PARA CONDUZIR PA...,HUGO MARCONDES DE A. SANTANA ...,008.080.355-50,"100,00",10 - SAUDE ...,122 - ADMINISTRACAO GERAL ...,339014000000000000 - Diarias-Civil ...,0002 - REC.IMP.TRANSF.IMP.SAUDE 15 ...,Isento
2446,06/07/2020,PAGAMENTO,5896,,REFERENTE FOLHA DE PAGAMENTO AUXILIO DOENCA EN...,FOLHA PAG. ENDEMIAS ESTATUTARI ...,08.576.590/0001-07,"39,20",10 - SAUDE ...,305 - VIGILIANCIA EPIDEMIOLOGICA ...,319011010000000000 - V.Vant.Fixas P.CivilVe.Ba...,0002 - REC.IMP.TRANSF.IMP.SAUDE 15 ...,Isento


In [67]:
cityhall_df.shape

(2721, 13)

Uma amostra dos dados do TCM-BA:

In [68]:
tcm_df.sample(3)

,NumeroDocumeto,empenho,data,municipio,unidade,cd_Unidade,orgao,unidadeOrcamentaria,credor,valor,cd_Orgao,cd_UnidadeOrcamentaria,dataEmpenho
830,06134-20-FMS,20000257,2020-07-10 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,BIOCENTER CENT HEM.E PAT.CLI.L,16536.39,11,1111,2020-01-02 00:00:00.000
1178,08542-20-PMFS,20000015,2020-07-16 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SEC.DE COMUNICACAO SOCIAL,SECRETARIA DE COMUNIC.SOCIAL,MERCADO DE PRO.E MARK. LTDA,1980.00,7,707,2020-02-17 00:00:00.000
561,06074-20-FMS,20001403,2020-07-08 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,IMAPS-INST.MARIE PIERRE DE SAUDE ASSOC. PROT. MAT,43790.38,11,1111,2020-03-03 00:00:00.000


In [69]:
tcm_df.shape

(2143, 13)

A partir do número de linhas podemos ver que os números não batem.
Mas vamos tentar cruzar os códigos de empenho para ver o que encontramos.

Antes de começarmos a explorar um pouco mais, vamos mapear os campos de cada arquivo:

| Campo (o que é) | Prefeitura | TCM-BA        | Observações |
| ------------- | ------------- | ------------- | ------------- |
| Data | `Data de Publicacao` | `data` | Assumindo que essa é a data do pagamento e não a data do empenho |
| Fase da despesa | `Fase` |  | |
| Número da despesa | `Numero` | `empenho` | |
| Número do processo | `N do processo` | `NumeroDocumeto` | Necessário confirmar |
| Descrição da despesa | `Bem / Servico Prestado` |  | |
| Credor | `Credor` | `credor` | |
| CPF/CNPJ do credor | `CPF / CNPJ` | | Presente apenas na página de detalhes |
| Valor | `Valor` | `valor` | |
| Função | `Funcao` | | |
| Subfunção | `Subfuncao` | | |
| Natureza | `Natureza` | | |
| Fonte | `Fonte` | | |
| Modalidade | `Modalidade` |  | |


## Match dos pagamentos

Infelizmente não temos tantos campos assim para fazer o _match_ das despesas.
Mas podemos tentar a partir do número da despesa e o número do processo.

Em alguns números das despesas nos dados da prefeitura não vem com o zero na frente
e nem com o ano da despesa. Por exemplo: `6848` na prefeitura e `06848-20-FMS` no TCM.
Pelo padrão, os números da despesa são formados de 5 dígitos, acrescidos do ano (`\d{5}-\d{2}-\w+`).
Podemos padronizar os que não atendem esse padrão em uma nova coluna. O problema é inferir o ano,
pois não temos como saber se todas as despesas foram apenas desse ano.

O número da despesa, representado por `NumeroDocumeto` no TCM-BA, vem seguido do órgão.
Exemplo: `07613-20-PMFS`. Precisamos cortar essa parte para bater com o número nos dados da prefeitura.

In [70]:
cityhall_df["Numero"]

0       09149-20                      
1       09150-20                      
2       3107                          
3       3106                          
4       3105                          
                     ...              
2716    08717-20                      
2717    08715-20                      
2718    08888-20                      
2719    08889-20                      
2720    09910-20                      
Name: Numero, Length: 2721, dtype: object

In [71]:
tcm_df["NumeroDocumeto"]

0        05705-20-FMS
1        05734-20-FMS
2        05716-20-FMS
3        05744-20-FMS
4        05714-20-FMS
            ...      
2138    09900-20-PMFS
2139    09901-20-PMFS
2140     06848-20-FMS
2141     06847-20-FMS
2142     06844-20-FMS
Name: NumeroDocumeto, Length: 2143, dtype: object

In [72]:
def split_number_from_agency(value):
    result = value["NumeroDocumeto"].rsplit("-", maxsplit=1)
    number = None
    agency = None
    if len(result) == 2:
        number = result[0]
        agency = result[1]
    if len(result) == 1:
        number = result[0]
    return pd.Series([number, agency],index=['numeroProcesso','Orgao'])


tcm_df[["numeroProcesso", "Orgao"]] = tcm_df.apply(split_number_from_agency, axis=1)
tcm_df.sample(5)

,NumeroDocumeto,empenho,data,municipio,unidade,cd_Unidade,orgao,unidadeOrcamentaria,credor,valor,cd_Orgao,cd_UnidadeOrcamentaria,dataEmpenho,numeroProcesso,Orgao
1058,08436-20-PMFS,20000001,2020-07-16 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SEC.DESENVOLVIMENTO ECONOMICO,SEC.M.TRAB.TUR.DESENV ECONOM,TELEMAR NORTE LESTE S/A,131.46,14,1414,2020-01-02 00:00:00.000,08436-20,PMFS
1797,06533-20-FMS,20002062,2020-07-24 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,ANTENOR FRANCISCO DA SILVA,50.00,11,1111,2020-06-05 00:00:00.000,06533-20,FMS
1293,09080-20-PMFS,20000686,2020-07-17 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SEC.DESENVOLVIMENTO SOCIAL,UNIDADE GESTORA DO FMAS,I.N.S.S. - FOLHA,2665.77,12,1224,2020-04-01 00:00:00.000,09080-20,PMFS
1056,06543-20-FMS,20000666,2020-07-15 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,TELEMAR NORTE LESTE S/A,140.89,11,1111,2020-01-02 00:00:00.000,06543-20,FMS
818,06138-20-FMS,20000281,2020-07-10 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,HL DE OLIVEIRA NETO E CIA LTDA,78702.36,11,1111,2020-01-02 00:00:00.000,06138-20,FMS


Hora de verificar os encontrados! Como o número de despesas da prefeitura é maior do que
o número de despesas no TCM-BA vamos utilizar os dados da prefeitura como base.

In [97]:
def normalize_process_number(value):
    # TODO converter para inteiro, para facilitar o merge
    # 06848/6848/06848-20-PMS para 6848
    pass

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2143 entries, 0 to 2142
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NumeroDocumeto          2143 non-null   object 
 1   empenho                 2143 non-null   int64  
 2   data                    2143 non-null   object 
 3   municipio               2143 non-null   object 
 4   unidade                 2143 non-null   object 
 5   cd_Unidade              2143 non-null   int64  
 6   orgao                   2143 non-null   object 
 7   unidadeOrcamentaria     2143 non-null   object 
 8   credor                  2143 non-null   object 
 9   valor                   2143 non-null   float64
 10  cd_Orgao                2143 non-null   int64  
 11  cd_UnidadeOrcamentaria  2143 non-null   int64  
 12  dataEmpenho             2143 non-null   object 
 13  numeroProcesso          2143 non-null   object 
 14  Orgao                   2141 non-null   

In [98]:
merged_df = pd.merge(cityhall_df, tcm_df, how='inner', left_on=["Numero"], right_on=["numeroProcesso"], suffixes=('_prefeitura', '_tcmba'))
merged_df

,Data de Publicacao,Fase,Numero,N do processo,Bem / Servico Prestado,Credor,CPF / CNPJ,Valor,Funcao,Subfuncao,...,cd_Unidade,orgao,unidadeOrcamentaria,credor,valor,cd_Orgao,cd_UnidadeOrcamentaria,dataEmpenho,numeroProcesso,Orgao


Infelizmente não encontramos nada (devido a um bug no pandas). #FIXME

Vamos dar uma olhada em como estão os dados do número da despesa na prefeitura.

In [74]:
cityhall_df["Numero"].unique()

array(['09149-20                      ', '09150-20                      ',
       '3107                          ', ...,
       '08888-20                      ', '08889-20                      ',
       '09910-20                      '], dtype=object)

In [75]:
tcm_df["numeroProcesso"].unique()

array(['05705-20', '05734-20', '05716-20', ..., '06848-20', '06847-20',
       '06844-20'], dtype=object)

Vamos tentar pela data e pelo valor das despesas.

In [99]:
import dateparser


cityhall_df["Data de Publicacao"].apply(dateparser.parse)

ModuleNotFoundError: No module named 'dateparser'

In [76]:
from scripts.parsers import currency_to_float


cityhall_df["Valor"] = cityhall_df["Valor"].apply(currency_to_float)

In [77]:
cityhall_df["Valor"]

0        1071.80
1         543.38
2         776.00
3         240.00
4       18549.76
          ...   
2716     -656.00
2717      -38.00
2718        5.50
2719        5.50
2720      330.00
Name: Valor, Length: 2721, dtype: float64

In [81]:
merged_by_value_df = pd.merge(cityhall_df, tcm_df, how='inner', left_on=["Valor"], right_on=["valor"], suffixes=('_prefeitura', '_tcmba'))
merged_by_value_df.sample(5)

,Data de Publicacao,Fase,Numero,N do processo,Bem / Servico Prestado,Credor,CPF / CNPJ,Valor,Funcao,Subfuncao,...,cd_Unidade,orgao,unidadeOrcamentaria,credor,valor,cd_Orgao,cd_UnidadeOrcamentaria,dataEmpenho,numeroProcesso,Orgao
2651,23/07/2020,PAGAMENTO,6505,,REFERENTE A VIAGEM A SALVADOR PARA BUSCAR O PA...,GILVAN PEREIRA SOARES ...,385.436.055-04,50.00,10 - SAUDE ...,122 - ADMINISTRACAO GERAL ...,...,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,GEORGENY RIBEIRO CARVALHO,50.00,11,1111,2020-06-18 00:00:00.000,06529-20,FMS
1027,24/07/2020,PAGAMENTO,6521,,REFERENTE A VIAGEM A SALVADOR PARA BUSCAR A PA...,DANIEL OLIMPIO DOS SANTOS ...,375.242.165-72,50.00,10 - SAUDE ...,122 - ADMINISTRACAO GERAL ...,...,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,JOSE ENOCK DA CRUZ JUNIOR,50.00,11,1111,2020-07-02 00:00:00.000,06428-20,FMS
7543,08/07/2020,PAGAMENTO,08062-20,,REF A DESPESA COM CONSUMO DE ÁGUA.PERÍODO: 06/...,EMBASA EMP BAIANA DE A E SANEA ...,13.504.675/0001-10,105985.94,12 - EDUCACAO ...,122 - ADMINISTRACAO GERAL ...,...,129,SECRETARIA DE EDUCACAO,FUNDO MUNICIPAL DE EDUCACAO,EMBASA EMP BAIANA DE A E SANEA,105985.94,9,909,2020-01-02 00:00:00.000,08062-20,PMFS
2366,23/07/2020,PAGAMENTO,6490,,REFERENTE A VIAGEM A SALVADOR PARA BUSCAR O PA...,ROBERTO CARLOS DE JESUS ...,461.776.205-15,50.00,10 - SAUDE ...,122 - ADMINISTRACAO GERAL ...,...,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,EVANDO LIMA DOS SANTOS,50.00,11,1111,2020-06-30 00:00:00.000,06481-20,FMS
5241,08/07/2020,PAGAMENTO,08059-20,353-2015,"REF. DESPESAS COM 47° MEDIÇÃO, REFERENTE À EX...",STE SERVIÇOS TÉCNICOS DE ENGENHARIA SA ...,88.849.773/0001-98,101618.89,04 - ADMINISTRACAO ...,121 - PLANEJAMENTO E ORCAMENTO ...,...,129,SECRETARIA DE PLANEJAMENTO,SECRETARIA DE PLANEJAMENTO,STE SERVIÇOS TÉCNICOS DE ENGENHARIA SA,101618.89,8,808,2020-04-22 00:00:00.000,08908-20,PMFS


In [100]:
fields = [
    "Data de Publicacao", "data",
    "Numero", "NumeroDocumeto",
    "N do processo", "numeroProcesso",
    "Credor", "credor",
    "Valor", "valor",
]

merged_by_value_df[fields]

,Data de Publicacao,data,dataEmpenho,Numero,NumeroDocumeto,N do processo,numeroProcesso,Credor,credor,Valor,valor
0,31/07/2020,2020-07-02 00:00:00.000,2020-01-02 00:00:00.000,3106,07717-20-PMFS,118-2020-1123D,07717-20,AL REIS COMERCIO E SERVICO LTDA-ME ...,AUTO CENTRO AUTOMOTIVO OLIVEIRA EIRELI,240.00,240.00
1,31/07/2020,2020-07-31 00:00:00.000,2020-01-02 00:00:00.000,6847,06847-20-FMS,45-2017-11D,06847-20,BANCO DO BRASIL S/A CENTRO ...,BANCO DO BRASIL S/A CENTRO,949.75,949.75
2,31/07/2020,2020-07-31 00:00:00.000,2020-01-02 00:00:00.000,6844,06844-20-FMS,,06844-20,CAIXA ECONOMICA FEDERAL - ...,CAIXA ECONOMICA FEDERAL -,190.58,190.58
3,31/07/2020,2020-07-31 00:00:00.000,2020-01-02 00:00:00.000,6848,06848-20-FMS,45-2017-11D,06848-20,BANCO DO BRASIL S/A CENTRO ...,BANCO DO BRASIL S/A CENTRO,623.80,623.80
4,31/07/2020,2020-07-31 00:00:00.000,2020-04-01 00:00:00.000,09169-20,09169-20-PMFS,,09169-20,TRIBUNAL R.DO TRAB.DA 5.REGIAO ...,TRIBUNAL R.DO TRAB.DA 5.REGIAO,131788.71,131788.71
...,...,...,...,...,...,...,...,...,...,...,...
7821,01/07/2020,2020-07-01 00:00:00.000,2020-01-20 00:00:00.000,08715-20,08715-20-PMFS,,08715-20,EDNALVA VITORIO DE SOUZA ...,EDNALVA VITORIO DE SOUZA,-38.00,-38.00
7822,01/07/2020,2020-07-01 00:00:00.000,2020-03-02 00:00:00.000,08888-20,08888-20-PMFS,,08888-20,BANCO DO BRASIL S/A CENTRO ...,BANCO DO BRASIL S/A CENTRO,5.50,5.50
7823,01/07/2020,2020-07-01 00:00:00.000,2020-03-02 00:00:00.000,08888-20,08889-20-PMFS,,08889-20,BANCO DO BRASIL S/A CENTRO ...,BANCO DO BRASIL S/A CENTRO,5.50,5.50
7824,01/07/2020,2020-07-01 00:00:00.000,2020-03-02 00:00:00.000,08889-20,08888-20-PMFS,,08888-20,BANCO DO BRASIL S/A CENTRO ...,BANCO DO BRASIL S/A CENTRO,5.50,5.50
